In [1]:
import argparse
import os
import copy
import matplotlib

matplotlib.use('agg')
import matplotlib.pyplot as plt
import numpy as np

from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.backends import cudnn
import torchvision
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST

from model import *
from progressBar import printProgressBar
from utils import *

from time import time

parser = argparse.ArgumentParser() 
outd = 'Results'#dhdh# GAN result 저장소
outf ='Images' 
outl='Losses' #dhdh# loss
outm='Models' #dhdh# model weight 저장소

workers=8
batchSizes=[16, 16, 16, 16, 8, 8]
nch=4
BN=True
WS=True
PN=True

n_iter=4
lambdaGP=10
gamma=1
e_drift=0.001
saveimages=1
savenum=64
savemodel=10
savemaxsize= True
# opt = parser.parse_args()

In [2]:
!opt = parser.parse_args()

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `opt = parser.parse_args()'


In [3]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MAX_RES = 5 ## for 128x128 output (2^2 * 2^MAX_RES) 이므로

transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

dataset = torchvision.datasets.ImageFolder(root='/home/capstone_ai1/kong/403_High_Small/',
                                        transform=transform,
                                        )

In [4]:
if not os.path.exists(outd):
    os.makedirs(outd)    
for f in [outf, outl, outm]:
    if not os.path.exists(os.path.join(outd, f)):
        os.makedirs(os.path.join(outd, f))       ## 저장소 생성 코드

In [5]:
G = Generator(max_res=MAX_RES, nch=nch, nc=3, bn=BN, ws=WS, pn=PN).to(DEVICE)
D = Discriminator(max_res=MAX_RES, nch=nch, nc=3, bn=BN, ws=WS).to(DEVICE)

#dhdh# Generator, Discriminator 생성

#dhdh# max_res : 최종 생성이미지 크기
#dhdh# nch : latent space feature map 크기 = nch *32 ex) nch가 16이면 f_m 크기=512
#dhdh# nc : input channel
#dhdh# bn : if True, BatchNormalization 실행
#dhdh# ws : if True, Weight Scaling 실행 : parameter weight들이 동일한 dynamic range를 갖도록 해줌.
#dhdh# pn : pixel normalization (generator에만 쓰임)

In [6]:
if not WS:
    # weights are initialized by WScale layers to normal if WS is used
    G.apply(weights_init)
    D.apply(weights_init)
Gs = copy.deepcopy(G) #dhdh# save test용으로 복사
optimizerG = Adam(G.parameters(), lr=1e-3, betas=(0, 0.99))
optimizerD = Adam(D.parameters(), lr=1e-3, betas=(0, 0.99))

GP = GradientPenalty(batchSizes[0], lambdaGP, gamma, device=DEVICE)
#dhdh# GradientPenalty
# Computes the gradient penalty as defined in "Improved Training of Wasserstein GANs
# (https://arxiv.org/abs/1704.00028)



In [7]:
epoch = 0
global_step = 0
total = 2
d_losses = np.array([])
d_losses_W = np.array([])
g_losses = np.array([])

In [8]:
P = Progress(n_iter, MAX_RES, batchSizes)
##dhdh## n_iter (int): the number of epochs before changing the progress,
##dhdh## pmax (int): the maximum progress of the training.

In [9]:
##dhdh## Progress Class 분석

class Progress:
    def __init__(self, n_iter, pmax, batchSizeList):
        assert n_iter > 0 and isinstance(n_iter, int), 'n_iter must be int >= 1'
        assert pmax >= 0 and isinstance(pmax, int), 'pmax must be int >= 0'
        assert isinstance(batchSizeList, list) and \
               all(isinstance(x, int) for x in batchSizeList) and \
               all(x > 0 for x in batchSizeList) and \
               len(batchSizeList) == pmax + 1, \
            'batchSizeList must be a list of int > 0 and of length pmax+1'
        ##dhdh## 위 규칙을 안 지킬 시 오류 발생
        self.n_iter = n_iter
        self.pmax = pmax
        self.p = 0
        self.batchSizeList = batchSizeList

    def progress(self, epoch, i, total):
        """Update the progress given the epoch and the iteration of the epoch
        Args:
            epoch (int): batch of images to resize
            i (int): iteration in the epoch
            total (int): total number of iterations in the epoch
        """
        ##dhdh##
        
        ## ex) epoch = 8, total = 2000, i =20, n_iter = 4
        x = (epoch + i / total) / self.n_iter
        ## x = 8.001/4 = 2.000025
        self.p = min(max(int(x / 2), x - ceil(x / 2), 0), self.pmax)
        ## p = min(max(1 , 0.000025 , 0), pmax = 8 )  -> p = 1
        
        return self.p

    def resize(self, images):
        """Resize the images  w.r.t the current value of the progress.
        Args:
            images (Variable or Tensor): batch of images to resize
        """
        ##dhdh##  progress에 맞게 resize
        x = int(ceil(self.p))
        if x >= self.pmax: 
            return images
        else:
            return F.adaptive_avg_pool2d(images, 4 * 2 ** x)

    @property
    def batchSize(self):
        """Returns the current batchSize w.r.t the current value of the progress"""
        x = int(ceil(self.p))
        return self.batchSizeList[x]
    


In [10]:
z_save = hypersphere(torch.randn(savenum, nch * 32, 1, 1, device=DEVICE))

##dhdh## 나중에 GAN 학습결과 이미지를 생성할때 쓰일 latent vectors 생성
##shape = 64 * 128 * 1 * 1
z_save.shape

torch.Size([64, 128, 1, 1])

In [11]:
P.progress(epoch, 1, total)
GP.batchSize = P.batchSize
# Creation of DataLoader
data_loader = DataLoader(dataset,
                         batch_size=P.batchSize,
                         shuffle=True,
                         num_workers=workers,
                         drop_last=True,
                         pin_memory=True)

In [12]:
## While True:

t0 = time()

lossEpochG = []
lossEpochD = []
lossEpochD_W = []

G.train()
cudnn.benchmark = True

P.progress(epoch, 1, total)

0

In [16]:
if P.batchSize != data_loader.batch_size:
    # update batch-size in gradient penalty
    GP.batchSize = P.batchSize
    # modify DataLoader at each change in resolution to vary the batch-size as the resolution increases
    data_loader = DataLoader(dataset,
                             batch_size=P.batchSize,
                             shuffle=True,
                             num_workers=workers,
                             drop_last=True,
                             pin_memory=True)
total = len(data_loader)

In [43]:
images = enumerate(data_loader)
i,(images,_) = next(images)

In [48]:
P.progress(epoch, i + 1, total + 1)
if P.batchSize != data_loader.batch_size:
    # update batch-size in gradient penalty
    GP.batchSize = P.batchSize
    # modify DataLoader at each change in resolution to vary the batch-size as the resolution increases
    data_loader = DataLoader(dataset,
                             batch_size=P.batchSize,
                             shuffle=True,
                             num_workers=opt.workers,
                             drop_last=True,
                             pin_memory=True)
total = len(data_loader)

In [56]:
images = enumerate(data_loader)
i,(images,_) = next(images)
P.progress(epoch, i + 1, total + 1) ##dhdh# return self.p = 0
global_step += 1
images = images.to(DEVICE)

In [57]:
images.shape

torch.Size([16, 3, 128, 128])

In [59]:
images = P.resize(images) ##dhdh## self.p 가 0 이기 때문에, shape를 4,4로 resize
images.shape

torch.Size([16, 3, 4, 4])

In [61]:
D.zero_grad()
z = hypersphere(torch.randn(P.batchSize, nch * 32, 1, 1, device=DEVICE))
z.shape

torch.Size([16, 128, 1, 1])

In [62]:
with torch.no_grad():
    fake_images = G(z, P.p) ##dhdh## 4x4 사이즈 fake_image 생성

In [68]:
D_real = D(images, P.p)
D_realm = D_real.mean()
D_fake = D(fake_images, P.p)
D_fakem = D_fake.mean()

In [69]:
D_realm

tensor(-0.0605, device='cuda:0', grad_fn=<MeanBackward0>)

In [73]:
# compute gradient penalty for WGAN-GP as defined in the article
gradient_penalty = GP(D, images.data, fake_images.data, P.p)

# prevent D_real from drifting too much from 0
drift = (D_real ** 2).mean() * e_drift

In [74]:
d_loss = D_fakem - D_realm
d_loss_W = d_loss + gradient_penalty + drift
d_loss_W.backward()
optimizerD.step()

lossEpochD.append(d_loss.item())
lossEpochD_W.append(d_loss_W.item())

In [78]:
# =============== Train the generator ===============#
G.zero_grad()

z = hypersphere(torch.randn(P.batchSize, nch * 32, 1, 1, device=DEVICE))
fake_images = G(z, P.p)

# compute scores with new fake images
G_fake = D(fake_images, P.p)
G_fakem = G_fake.mean()
# no need to compute D_real as it does not affect G
g_loss = -G_fakem
# Optimize
g_loss.backward()
optimizerG.step()

In [80]:
lossEpochG.append(g_loss.item())
# update Gs with exponential moving average
exp_mov_avg(Gs, G, alpha=0.999, global_step=global_step)

printProgressBar(i + 1, total + 1,
                         length=20,
                         prefix=f'Epoch {epoch} ',
                         suffix=f', d_loss: {d_loss.item():.3f}'
                                f', d_loss_W: {d_loss_W.item():.3f}'
                                f', GP: {gradient_penalty.item():.3f}'
                                f', progress: {P.p:.2f}')
printProgressBar(total, total,
                     done=f'Epoch [{epoch:>3d}]  d_loss: {np.mean(lossEpochD):.4f}'
                          f', d_loss_W: {np.mean(lossEpochD_W):.3f}'
                          f', progress: {P.p:.2f}, time: {time() - t0:.2f}s'
                     )

Epoch [  0]  d_loss: 0.0109, d_loss_W: 2.632, progress: 0.00, time: 1938.70s                                 


In [83]:
d_losses = np.append(d_losses, lossEpochD)
d_losses_W = np.append(d_losses_W, lossEpochD_W)
g_losses = np.append(g_losses, lossEpochG)

# np.save(os.path.join(optoutd, opt.outl, 'd_losses.npy'), d_losses)
# np.save(os.path.join(opt.outd, opt.outl, 'd_losses_W.npy'), d_losses_W)
# np.save(os.path.join(opt.outd, opt.outl, 'g_losses.npy'), g_losses)

cudnn.benchmark = False ##dhdh## True로 하면 input tensor를 하드웨어에 맞게 최적화 해줌
# 그러나 input tenor의 사이즈가 계속 바뀌면 악영향을 끼침.

